#  주택청약 FAQ 시스템 챗봇 구현 
- 문서 전처리 + RAG + Gradio ChatInterface

### **학습 목표:**  RAG 기반의 주택청약 FAQ 시스템을 Gradio로 구현한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [13]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.9, 
)

# **문서 전처리 파이프라인**

* 문서 전처리의 첫 단계는 데이터 정제로, 원본 문서에서 불필요한 요소(HTML 태그, 특수문자, 중복 내용 등)를 제거하고 텍스트를 표준화하는 과정입니다. 이는 검색 품질과 직결되는 중요한 단계입니다.

* 문서 청킹(Chunking)은 긴 문서를 의미 있는 작은 단위로 분할하는 과정으로, 문장 단위나 단락 단위로 나누되 문맥의 연속성을 유지하는 것이 핵심입니다. 이는 검색 정확도와 답변 생성의 품질에 직접적인 영향을 미칩니다.

* 임베딩(Embedding) 생성은 텍스트를 고차원의 벡터로 변환하는 과정으로, 문서의 의미적 특성을 수치화하여 효율적인 검색을 가능하게 합니다. 이때 사용되는 임베딩 모델의 선택이 검색 성능을 좌우하는 중요한 요소가 됩니다.

* 마지막으로 벡터 데이터베이스 색인화 단계에서는 생성된 임베딩을 효율적으로 저장하고 검색할 수 있는 구조로 변환합니다. 이는 대규모 문서 집합에서도 빠른 검색을 가능하게 하는 핵심 요소입니다.


### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공
- Q1 ~ Q50까지 모두 50개의 문답이 포함된 텍스트 파일

In [ ]:
# 파일 경로 설정
faq_text_file = "data/housing_faq.txt"

# 파일 읽기 - 파이썬 내장 함수 사용
### 윈도우의 경우 utf-8로 인코딩 필요 
with open(faq_text_file, 'r', encoding="utf-8") as f:
    faq_text = f.read()

# 파일 내용 확인
print(faq_text[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


##### ***[실습] TextLoader를 사용하여, 텍스트 문서를 로드합니다.*** 

In [ ]:
# 여기에 코드를 작성하세요.
from langchain_community.document_loaders import TextLoader

### 파일 로더 초기화 
txt_loader = TextLoader("./data/housing_faq.txt", encoding="utf-8")

### 로딩 
docs = txt_loader.load()

In [6]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [7]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': './data/housing_faq.txt'}

### 2) 문서 전처리

`(1) 각 질문과 답변을 쌍으로 추출하여 정리 (정규표현식 활용)`
- FAQ이기 때문에 Q1, Q2 처럼 정규식으로 처리가 가능 

In [8]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [9]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


`(2) LLM으로 추가 정보를 추출`
- 텍스트에서 키워드와 핵심 개념을 추출하는 체인
- 메타데이터 or 본문(page_content)에 추가하여 검색에 활용    

##### ***[실습] 다음 모델 스키마를 사용하여, 키워드와 요약을 추출하는 체인을 완성합니다.*** 

In [10]:
# QA 쌍 확인 (첫번째)
print((qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer']))

경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?

해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 텍스트 분석 전문가입니다. 
주어진 텍스트에서 중요한 키워드를 추출하고, 텍스트의 간단한 요약을 작성하는 것이 당신의 역할입니다.

## 추출 지침:
- 텍스트의 맥락을 고려하여 핵심 용어나 전문 용어를 추출합니다
- 주요 아이디어나 원리, 개념을 포함합니다
- 가장 중요한 키워드를 1개 추출합니다
- 요약은 1문장으로 간결하게 작성합니다

## 출력 형식:
- keyword: 가장 중요한 키워드 
- summary: 텍스트의 간단한 요약"""),
    
    ("user", "다음 텍스트를 분석해주세요:\n\n{input_text}")
])

# LCEL 체인 구성
llm_with_structure = llm.with_structured_output(KeywordOutput) 
keyword_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyword_extractor.invoke(qa_pairs[0]['question']+qa_pairs[0]['answer'])
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 해당 주택건설지역
요약: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역이며, 특별시·광역시·특별자치시·특별자치도 또는 시·군의 행정구역을 의미한다.


`(3) QA 쌍을 문자열 포맷팅하고 문서 객체로 변환`

In [24]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # QA 쌍을 포맷팅
        formatted_output = (
            f"[{pair['number']}]\n"
            f"질문: {pair['question']}\n"
            f"답변: {pair['answer']}\n"
        )

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=formatted_output,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1,
 'summary': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.'}


##### ***[실습] 문서 객체를 포맷팅하여 구성합니다.*** 

- 요약문을 시맨틱 검색에 활용합니다. 다음 구조로 문서 객체를 생성합니다. 
    - page_content: 요약
    - metadata: 기타 정보

In [25]:
# 여기에 코드를 작성하세요.
from langchain_core.documents import Document

def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # 키워드와 요약 추출
        result = keyword_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
            }
        )
        processed_docs.append(doc)

    return processed_docs


In [26]:
# QA 쌍 포맷팅
summary_formatted_docs = format_qa_pairs_with_summary(qa_pairs) 
print(f"포맷팅된 문서 개수: {len(summary_formatted_docs)}")

# 문서 확인
print(summary_formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

포맷팅된 문서 개수: 50
경기도 과천시에서 공급되는 주택의 해당 주택건설지역은 과천시 행정구역 전체를 의미한다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '해당 주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1}


# 벡터 저장 

In [28]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장
vector_store = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_db",
    persist_directory="./chroma_db",
)

c:\Modu_LLM\LLM_faq_chatbot_tutorial\faq_chatbot\.venv\Lib\site-packages\chromadb\execution\expression\operator.py:239: SyntaxWarning: invalid escape sequence '\.'
  Key("email").regex(r".*@example\.com")


##### ***[실습] 요약 문서(summary_formatted_docs)를 벡터 스토어에 저장합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [30]:
print(formatted_docs[0].page_content)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.



In [32]:
vector_store._collection.count()

50

In [33]:
# 여기에 코드를 작성하세요.
# 문서 저장
output_file = "data/housing_faq_formatted.json"

with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

포맷팅된 문서를 data/housing_faq_formatted.json에 저장했습니다.


# 문서 검색

In [34]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

vector_store._collection.count()

50

##### ***[실습] 앞에서 저장한 벡터 스토어를 로드합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [ ]:
# 여기에 코드를 작성하세요.


# 벡터 저장소 로드
vector_store_with_summary = None

In [ ]:
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

##### ***[실습] MMR 검색기를 정의합니다.*** 

- 요약 문서 벡터 스토어를 사용
- 10개의 문서를 가져와서, 다양성 기반으로 3개를 선택 (다양성은 중간 수준 적용)
- jsonl 기준으로

In [36]:
# 여기에 코드를 작성하세요.

mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 8, "lambda_mult": 0.5 }
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1
경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원
47
무주택세대구성원이란?
무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.
[36]
질문: 청년주택드림청약통장 가입(혹은 전환신규) 당시 무주택자 였는데 이후 주택을 소유하게 된 경우 우대이율 및 비과세 적용을 받을 수 있나요?
답변: 청년주택드림통장의 우대이율요건과 비과세 요건은 동일하지 않습니다. 비과세 요건 중 ‘주택을 소유하지 않은 세대의 세대주’ 자격은 가입 당시 기준입니다. 따라서 이후 주택을 소유하더라도 비과세 적용

### **[심화] 메타데이터 기반 필터링**

- Chroma 문서: https://docs.trychroma.com/docs/querying-collections/metadata-filtering

In [38]:
# 단일 필드 정확히 일치
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": "해당 주택건설지역"}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1


- $eq 연산자 사용 - 정확히 일치

In [ ]:
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$eq": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1


- `$ne` (Not Equal) 연산자 사용 - 정확히 일치하지 않는 문서 검색

In [ ]:
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$ne": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

--------------------------------------------------
청약신청
2
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원
47
[7]
질문: 행정중심복합도시예정지역에서 공급하는 주택의 경우 공급비율 및 대상은?
답변: 행정중심복합도시 예정지역에서 공급하는 주택의 경우 「주택공급에 관한 규칙」 제34조에 따라 해당 주택건설지역 거주자에게 행정중심복합도시건설청장이 정하여 고시하는 비율(현행 60%)을 우선공급하고 있으며, 이후 남은 물량은 「주택공급에 관한 규칙」 제4조제1항제3호 가목에 따라 해당 주택 건설지역에 거주하지 않는 자도 공급대상에 포함하여 공급하고 있습니다.

--------------------------------------------------
행정중심복합도시 주택 공급 비율
7
[6]
질문: 「주택공급에 관한 규칙」 제34조에 따른 대규모택지개발지구에서 주택이 공급되는 경우 일반공급 뿐만 아니라 특별공급 물량도 공급비율에 따라 배정되는지?
답변: 공급규칙 제34조가 적용되는 지역에 주택을 공급하는 경우 특별공급 물량 또한 그 공급 비율에 따라 배정하여야

- $in (Includes) 연산자 사용 - 특정 값이 포함된 문서 검색

In [43]:
# $in 연산자로 여러 값 중 일치하는 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$in": ["해당 주택건설지역", "청약예금"]}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1
[39]
질문: 인천광역시 거주자로 청약예금 400만원(전용면적 102㎡ 이하)에 가입한 자가 입주자 모집공고일 전 서울시로 이주한 경우 102㎡ 이하의 주택에 청약하려면?
답변: 서울의 경우 전용면적 102㎡ 이하 주택에 청약할 수 있는 예치금액은 600만원이기 때문에 청약접수 당일까지 부족금액인 200만원을 추가로 예치하여야만 102㎡ 이하의 주택에 청약이 가능

--------------------------------------------------
청약예금
39


- 숫자 범위 검색 $gt, $lt, $gte, $lte

In [44]:
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"question_id": {"$gte": 10}}},
)

query = "무주택자 기준은 무엇인가요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원
50
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원
47
[31]
질문: 청년주택드림청약통장의 소득공제 혜택은 기존 주택청약종합저축과 동일한가요?
답변: 현재 주택청약종합저축에서 제공하는 소득공제 조건(조세특례제한법 제87조)을 그대로 적용받게 되며, 연소득 7천만원 이하 무주택세대주로 무주택확인서를 제출하는 경우 연간 납입액 300만원 한도로 40%까지 소득공제가 가능합니다.

--------------------------------------------------
소득공제
31
[36]
질문: 청년주택드림청약통장 가입(혹은 전환신규) 당시 무주택자 였는데 이후 주택을 소유하게 된 경우 우대이율 및 비과세 적용을 받을 수 있나요?
답변: 청년주택드림통장의 우대이율요건과 비과세 요건은 동일하지 않습니다. 비과세 요건 중 ‘주택을 소유하지 않은 세대의 세대주’ 자격은 가입 당시 기준입니다. 따라서 이후 주택을 소유하더라도 비과세 적용 가능합니다. 다만, 우대이율의 경우 최초로 주택을 소유하게 된 년도의 직전년도 말까지만 우대이율을 적용받습니다. * 예시) ’21년에 가입 후 ’23년에 주택 구입 시 : ’22년 말까지 우대이율 적용

-------------------------------------

- $and 연산자 사용 - 여러 조건을 모두 만족하는 문서 검색

In [46]:
# $and로 여러 조건 조합 - keyword가 "주택건설지역"이고 question_id가 10 미만인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$and": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$lt": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1


- $or 연산자로 여러 조건 중 하나라도 일치하는 문서 검색

In [47]:
# $or로 여러 조건 중 하나 일치하는 문서 검색 - keyword가 "주택건설지역"이거나 question_id가 10 이상인 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$or": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$gte": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원
47
[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
무주택세대구성원
50
[38]
질문: 주민등록 거주지가 인천시인 청약신청자가 서울에서 분양하는 전용면적 85㎡이하의 민영주택을 청약코자 할 때 예치기준금액은?
답변: 주택공급에 관한 규칙 [별표2]에서 민영주택 청약 예치기준금액을 규정하고 있으며, 지역은 입주자모집공고일 현재 청약신청자의 주민등록표등초본상 거주지 기준, 예치 기준금액은 청약하고자 하는 주택의 평형을 기준으로 결정하여야 함. 따라서, 청약자가 인천시(그 밖의 광역시)에 거주하고 있고, 85㎡ 이하의 평형에 청약하는 경우 예치 기준금액

In [48]:
# 정규식 패턴 매칭 - page_content 본문에 "주택건설지역"이 포함된 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={'where_document': {'$contains': '해당 주택건설지역'}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
1
[2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

--------------------------------------------------
청약신청
2
[7]
질문: 행정중심복합도시예정지역에서 공급하는 주택의 경우 공급비율 및 대상은?
답변: 행정중심복합도시 예정지역에서 공급하는 주택의 경우 「주택공급에 관한 규칙」 제34조에 따라 해당 주택건설지역 거주자에게 행정중심복합도시건설청장이 정하여 고시하는 비율(현행 60%)을 우선공급하고 있으며, 이후 남은 물량은 「주택공급에 관한 규칙」 제4조제1항제3호 가목에 따라 해당 주택 건설지역에 거주하지 않는 자도 공급대상에 포함하여 공급하고 있습니다.

--------------------------------------------------
행정중심복합도

##### ***[실습] 메타데이터 필터링 조건을 적용하는 실습을 수행합니다..*** 

- 요약 문서 벡터 스토어 기반 MMR 검색기에 적용 

In [49]:
# 여기에 코드를 작성하세요.

mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, 
                   "fetch_k": 8, 
                   "lambda_mult": 0.5,
                   "filter": {"question_id": {"$gte": 10}}
                   }
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원
47
무주택세대구성원이란?
무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.
[38]
질문: 주민등록 거주지가 인천시인 청약신청자가 서울에서 분양하는 전용면적 85㎡이하의 민영주택을 청약코자 할 때 예치기준금액은?
답변: 주택공급에 관한 규칙 [별표2]에서 민영주택 청약 예치기준금액을 규정하고 있으며, 지역은 입주자모집공고일 현재 청약신청자의 주민등록표등초본상 거주지 기준, 예치 기준금액은 청약하고자 하는 주택의 평형을 기준으로 결정하여야 함. 따라서, 청약자가 인천시(그 밖의 광역시)에 거주하고 있고, 85㎡ 이하의 평형에 청약하는 경우 예치 기준금액은 250만원입니다. 민영주택 청약 예치기준금액 (제10조제1항 관련) (단위 : 만원) 지역 특별시 및 특별시 및 광역시를 공급받을 수 있는 그 밖의 광역시 주택의 전용면적 부산광역시 제외한 지역 85㎡ 이하 300 250 200 102㎡ 이하 600 400 300 135㎡ 이하 1,000 700 400 모든 면적 1,500 1,000 500 비고: “지역”은 입주자모집공고일 현재 주택공급신청자의 주민등록표에 따른 거주지역을 말한다

--------------------------------------------------
민영주택 청약 예치기준금액
38
주민등록 거주지가 인천시인 청약신청자가 서울에서 분양하는 전용면적 85㎡이하의 민영주택을 청약코자 할 때 예치기준금액은?
주택공급에 관한 규칙 [별표2]에서 민영주택 청약 예치기준금액을 규정하고 있으며, 지역은 입주자모집공고일 현재 청약신청자의 주민등록표등초본상 거주지 기준, 예치 기준금액은 청약하고자 하는 주택의 평형을 기준으로

In [57]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional

class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    lower_question_id: Optional[int] = Field(description="질문 ID 최소값($gt, $gte)")
    upper_question_id: Optional[int] = Field(description="질문 ID 최대값($lt, $lte)")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
다음 예시를 참조한다:

1. 키워드 검색 예시:
- 입력: "주택건설 관련 문서 찾아줘"
출력:
keyword: "주택건설"
keyword_expression: "$eq"

2. 질문 ID 검색 예시:
- 입력: "질문 ID 10번 이상인 문서"
출력:
lower_question_id: 10
question_id_expression: "$gte"

3. 복합 검색 예시:
- 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
출력:
keyword: "주택건설"
keyword_expression: "$eq"
lower_question_id: 5 
lower_question_id_expression: "$gte"
upper_question_id: 15
upper_question_id_expression: "$lte"

검색 표현식은 다음과 같이 사용한다:
- 동등 비교: $eq
- 크거나 같음: $gte
- 작거나 같음: $lte
- 범위 검색: $gt, $lt

요청에 해당 정보가 없는 경우 해당 필드는 null로 반환한다.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{query}")
])

# 구조화된 출력을 위한 체인 생성
model_with_structure = llm.with_structured_output(MetadataFilter)
metadata_chain = prompt | model_with_structure

# 사용 예시
query = "해당 주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"
filter_params = metadata_chain.invoke({"query": query})

print(f"키워드: {filter_params.keyword}")
print(f"키워드 표현식: {filter_params.keyword_expression}")
print(f"질문 ID: {filter_params.lower_question_id}")
print(f"질문 ID 표현식: {filter_params.question_id_expression}")

키워드: 주택건설지역
키워드 표현식: $eq
질문 ID: None
질문 ID 표현식: $lte


In [51]:
# 메타데이터 필터 생성
filter_dict = {}

if filter_params.keyword and filter_params.question_id:
    # 두 조건 모두 있는 경우 AND 연산자 사용
    filter_dict = {
        "$and": [
            {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
            {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
        ]
    }
elif filter_params.keyword:
    # 키워드 조건만 있는 경우
    filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
elif filter_params.question_id:
    # 질문 ID 조건만 있는 경우
    filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

# retriever에 필터 적용 
retriever = vector_store.as_retriever(
    search_kwargs={"filter": filter_dict} if filter_dict else {}
)

# 검색 실행
query = "주택건설지역 관련 문서를 질문 ID 10번 이하인 문서중에서 검색해주세요"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from pydantic import BaseModel, Field
from typing import Optional, Dict


class MetadataFilter(BaseModel):
    keyword: Optional[str] = Field(description="검색할 키워드")
    keyword_expression: Optional[str] = Field(description="키워드 검색 표현식")
    question_id: Optional[int] = Field(description="질문 ID")
    question_id_expression: Optional[str] = Field(description="질문 ID 검색 표현식")


def create_metadata_filter(query: str, llm) -> Dict:
    """
    사용자 쿼리를 분석하여 Chroma DB 검색을 위한 메타데이터 필터를 생성합니다.

    Args:
        query: 사용자 검색 쿼리.
        llm: 사용할 언어 모델.

    Returns:
        Chroma DB 검색에 사용할 수 있는 필터 딕셔너리.
    """
    system_prompt = """사용자 쿼리에서 키워드와 질문 ID 정보를 추출하여 Chroma DB 검색 필터를 생성한다.
    다음 예시를 참조한다:

    1. 키워드 검색 예시:
    - 입력: "주택건설 관련 문서 찾아줘"
    출력:
   {{keyword: "주택건설",keyword_expression: "$eq"}}
    
    2. 질문 ID 검색 예시:
    - 입력: "질문 ID 10번 이상인 문서"
    출력:
    
    question_id: 10
    question_id_expression: "$gte"

    3. 복합 검색 예시:
    - 입력: "주택건설 키워드가 있으면서 질문 ID가 5번에서 15번 사이인 문서"
    출력:
    keyword: "주택건설"
    keyword_expression: "$eq"
    question_id: 5
    question_id_expression: "$gte"

    검색 표현식은 다음과 같이 사용한다:
    - 동등 비교: $eq
    - 크거나 같음: $gte
    - 작거나 같음: $lte
    - 범위 검색: $gt, $lt

    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{query}")
    ])

    model_with_structure = llm.with_structured_output(MetadataFilter)
    metadata_chain = prompt | model_with_structure

    filter_params = metadata_chain.invoke({"query": query})

    filter_dict = {}

    if filter_params.keyword and filter_params.question_id:
        filter_dict = {
            "$and": [
                {"keyword": {filter_params.keyword_expression: filter_params.keyword}},
                {"question_id": {filter_params.question_id_expression: filter_params.question_id}}
            ]
        }
    elif filter_params.keyword:
        filter_dict = {"keyword": {filter_params.keyword_expression: filter_params.keyword}}
    elif filter_params.question_id:
        filter_dict = {"question_id": {filter_params.question_id_expression: filter_params.question_id}}

    return filter_dict



# 사용 예시 
query = "주택건설지역 관련 문서를 10번 이하인 문서중에서 검색해주세요"

@chain
def metadata_filter_query(query: str):

    llm = ChatOpenAI(
        model='gpt-4.1-mini',
        temperature=0.1,
        top_p=0.9,
    )

    filter_dict = create_metadata_filter(query, llm)
    print(filter_dict)

    retriever = vector_store.as_retriever(
            search_kwargs={"filter": filter_dict} if filter_dict else {}
    )
    results = retriever.invoke(query)

    return results

results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

{'$and': [{'keyword': {'$eq': '주택건설지역'}}, {'question_id': {'$lte': 10}}]}


In [53]:
# 다른 쿼리
query = "청약통장에 대한 정보를 찾아주세요"
results = metadata_filter_query.invoke(query)

for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

{'keyword': {'$eq': '청약통장'}}
[43]
질문: 청약통장이 압류 등으로 사용에 제한이 있거나, 청약통장 담보대출을 받고 있는 경우에도 해당 통장을 이용하여 당첨이 가능한지?
답변: 압류 등의 상태라고 하더라도 청약통장이 해지되지 않고 유지 중이라면 청약신청이 가능합니다. 청약통장 담보대출을 받은 경우에도 청약신청이 가능합니다. 다만, 담보대출 연체 등으로 해당 은행에서 청약통장을 해지하고 담보대출 상환처리하는 경우 새로 청약통장을 가입하여 청약 신청해야 하는 점 유의하시길 바랍니다.

--------------------------------------------------
청약통장
43


##### ***[실습] 메타데이터 필터링 체인의 성능을 개선합니다.*** 

- (예시)
    - 추가 예시 제공을 통해 필터링 적용 범위 확대
    - 복합조건식 추출 기능을 추가 등

In [ ]:
# 여기에 코드를 작성하세요.

# RAG Chain

### 1) 참조 문서 없이 직접 답변을 생성

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt
template = '''Answer the question based only on the following context.

[Context]
{context}

[Question]
{question}

[Answer (in 한국어)]
'''

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)


# Chain 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
rag_chain.invoke(query)

'수원시는 경기도에 속하는 시이므로, 수원시에서 공급되는 주택의 해당 주택건설지역은 수원시 행정구역 전체에 해당합니다.'

### 2) 참조 문서를 답변과 함께 반환

`(1) 문서와 포맷팅된 컨텍스트를 함께 반환하는 함수`


In [65]:
# 다음 코드를 완성하세요.

from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": question,  # 질문
        "context": format_docs(docs),   # 문서 포맷팅된 컨텍스트
        "source_documents": docs   # 검색된 문서 리스트
    }

`(2) 컨텍스트와 질문을 입력으로 받아 답변을 생성하는 함수`

In [66]:
# 다음 코드를 완성하세요.

from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = prompt | llm | StrOutputParser()
    answer = answer_chain.invoke(input_data)

    return {
        "answer": answer,  # 생성된 답변 (answer_chain 결과)
        "source_documents": input_data["source_documents"]  # 소스 문서 정보 (input_data에서 가져옴)
    }

`(3) RAG 체인 구성`

In [67]:
# 다음 코드를 완성하세요.

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    RunnableLambda(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': RunnableLambda(prompt_and_generate_answer),  # 답변 생성
        'question': itemgetter("question"),
        "source_documents": itemgetter("source_documents")  # 소스 반환
    }
)

In [68]:
# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

답변: 수원시는 경기도에 속하는 시이므로, 수원시에서 공급되는 주택의 해당 주택건설지역은 수원시 행정구역 전체에 해당합니다.

참조 문서:

문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


문서 2:
내용: [2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.


문서 3:
내용: [47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.



### 3) 검색 문서 관련성 평가

`(1) 검색 문서와 질문 간의 관련성을 평가`


In [69]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")


문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: no

문서 2:
내용: [2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

평가 결과: no

문서 3:
내용: [47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

평가 결과: no


In [70]:
# gpt-4.1 모델 사용

llm_gpt4o = ChatOpenAI(
    model='gpt-4.1',
    temperature=0.1,
    top_p=0.9, 
)

chain = prompt | llm_gpt4o | StrOutputParser()    # gpt-4.1 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")


문서 1:
내용: [1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

평가 결과: yes

문서 2:
내용: [2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

평가 결과: no

문서 3:
내용: [47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

평가 결과: no


##### ***[실습] 문서 관련성 평가 체인을 구조화 출력으로 구현합니다.*** 

- pydantic schema 사용
- with_structured_output 함수 사용

In [ ]:
# 여기에 코드를 작성하세요.

# Gradio 챗봇 인터페이스

In [71]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_sotre = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)


# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print("=" * 50)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

--------------------------------------------------
해당 주택건설지역
[2]
질문: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
답변: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

--------------------------------------------------
청약신청
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
무주택세대구성원


In [ ]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class SearchResult:
    context: str
    source_documents: Optional[List]

class RAGSystem:
    def __init__(
            self, 
            llm: BaseChatModel, 
            eval_llm: BaseChatModel,
            retriever: VectorStoreRetriever
        ):
        if not llm:
            self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        else:
            self.llm = llm

        if not eval_llm:
            self.eval_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
        else:
            self.eval_llm = eval_llm

        if not retriever:
            raise ValueError("검색기(retriever)가 필요합니다.")
        else:
            self.retriever = retriever
        
    def _format_docs(self, docs: List) -> str:
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _format_source_documents(self, docs: Optional[List]) -> str:
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            if 'summary' in metadata:
                source_info.append(f"요약: {metadata['summary']}")
                
            formatted_docs.append(
                f"📚 참조 문서 {i}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def _check_relevance(self, docs: List, question: str) -> List:
        """문서의 관련성 확인"""

        relevant_docs = []

        if not docs:
            return relevant_docs
            
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

        다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

        1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
        2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?

        'Yes' 또는 'No'로만 답변하세요."""),
            ("human", """[컨텍스트]
        {context}

        [질문]
        {question}""")
        ])
        
        chain = prompt | self.eval_llm | StrOutputParser()

        for doc in docs:
            result = chain.invoke({
                "context": doc.page_content,
                "question": question
            }).lower()

            print(f"문서 {doc.metadata['question_id']} 관련성 확인 결과: {result}")
            print(f"문서 {doc.metadata['question_id']} 내용:")
            print(doc.page_content)
            print("-" * 50)

            if "yes" in result:
                relevant_docs.append(doc)
            
        return relevant_docs
    
    def search_documents(self, question: str) -> SearchResult:
        try:
            docs = retriever.invoke(question)
            print(f"검색된 문서 개수: {len(docs)}")
            relevant_docs = self._check_relevance(docs, question) 
            print(f"관련 문서 개수: {len(relevant_docs)}")
            
            return SearchResult(
                context=self._format_docs(relevant_docs) if relevant_docs else "관련 문서를 찾을 수 없습니다.",
                source_documents=relevant_docs,
            )
        except Exception as e:
            print(f"문서 검색 중 오류 발생: {e}")
            return SearchResult(
                context="문서 검색 중 오류가 발생했습니다.",
                source_documents=None,
            )
    
    def generate_answer(self, message: str, history: List) -> str:
        # 문서 검색
        search_result = self.search_documents(message)
        
        if not search_result.source_documents:
            return "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"
                    
        # 프롬프트 템플릿 설정
        prompt = ChatPromptTemplate.from_messages([
            ("system", """다음 지침을 따라 질문에 답변해주세요:
            1. 주어진 문서의 내용만을 기반으로 답변하세요.
            2. 문서에 명확한 근거가 없는 내용은 "근거 없음"이라고 답변하세요.
            3. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 답변하세요.
            4. 추측이나 일반적인 지식을 사용하지 마세요."""),
            ("human", "문서들:\n{context}\n\n질문: {question}")
        ])
        
        # RAG Chain 구성
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            # 답변 생성
            answer = chain.invoke({
                "context": search_result.context,
                "question": message
            })
            
            # 참조 문서 포맷팅 추가
            sources = self._format_source_documents(search_result.source_documents)
            return f"{answer}\n{sources}"
            
        except Exception as e:
            return f"답변 생성 중 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정

rag_system = RAGSystem(
    llm=ChatOpenAI(model="gpt-4.1", temperature=0),   
    eval_llm=ChatOpenAI(model="gpt-4.1", temperature=0),
    retriever=vector_store.as_retriever(search_kwargs={"k": 2})
)

demo = gr.ChatInterface(
    fn=rag_system.generate_answer,
    title="RAG QA 시스템",
    description="""
    질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
    모든 답변에는 참조한 문서의 출처가 표시됩니다.
    """,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="gray",
    ),
examples=[
    ["수원시의 주택건설지역은 어디에 해당하나요?"],
    ["무주택 세대에 대해서 설명해주세요."],
    ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
],
)

# 데모 실행
demo.launch()

c:\Modu_LLM\LLM_faq_chatbot_tutorial\faq_chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Modu_LLM\LLM_faq_chatbot_tutorial\faq_chatbot\.venv\Lib\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


검색된 문서 개수: 3
문서 47 관련성 확인 결과: yes
문서 47 내용:
[47]
질문: 무주택세대구성원이란?
답변: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

--------------------------------------------------
문서 50 관련성 확인 결과: no
문서 50 내용:
[50]
질문: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
답변: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

--------------------------------------------------
문서 36 관련성 확인 결과: no
문서 36 내용:
[36]
질문: 청년주택드림청약통장 가입(혹은 전환신규) 당시 무주택자 였는데 이후 주택을 소유하게 된 경우 우대이율 및 비과세 적용을 받을 수 있나요?
답변: 청년주택드림통장의 우대이율요건과 비과세 요건은 동일하지 않습니다. 비과세 요건 중 ‘주택을 소유하지 않은 세대의 세대주’ 자격은 가입 당시 기준입니다. 따라서 이후 주택을 소유하더라도 비과세 적용 가능합니다. 다만, 우대이율의 경우 최초로 주택을 소유하게 된 년도의 직전년도 말까지만 우대이율을 적용받습니다. * 예시) ’21년에 가입 후 ’23년에 주택 구입 시 : ’22년 말까지 우대이율 적용

--------------------------------------------------
관련 문서 개수: 1


In [74]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7860


---

# **[실습] 주택청약 FAQ 시스템 구현**

### **문제 설명**
이전 코드를 기반으로 주택청약 FAQ 시스템을 다음 요구사항에 맞춰 개선합니다. 

1. 응답 품질 향상 (1개 이상)
   - 생성된 답변의 품질을 평가 (답변이 불충분한 경우 예외 처리)
   - 관련성 높은 FAQ 문서 검색 (임베딩 모델, 청크 크기, 벡터 검색 방법 등)

2. 사용자 경험 개선 (1개 이상)
   - 대화 이력 관리 기능 추가 (요약, 트리밍 기능 등 고려)
   - 최근 대화 기반 컨텍스트 구성 
   - 사용자 프로필 기반 맞춤 응답

### **제약 조건**
- Gradio ChatInterface 사용
- RAG 구조 유지

In [ ]:
# 여기에 코드를 작성하세요. 